In [1]:
%load_ext autoreload
%autoreload 1

In [20]:
%aimport bitcoin
import torch
import pyro
import time
import pyro.distributions as dist
import pyro.optim as optim
import pyro.poutine as poutine
import numpy as np
import logging
from torch.distributions import constraints
from bitcoin import BitcoinOTC
from pyro.infer import SVI, TraceGraph_ELBO

# Validation checks
logging.basicConfig(format='%(message)s', level=logging.INFO)
pyro.enable_validation(True)

In [3]:
data = bitcoin.BitcoinOTC()

c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:
omega_scale = 1.0
obs_scale = 1.0
embed_dim = 2

def guide(data, node_ind, edge_ind, edge_list):
    r"""Defines a variational family to use to fit an approximate posterior
    distribution for the probability model defined in model.
    """
    # Deleting arguments not used in the guide for linting purposes
    del edge_list
    
    # Parameters governing the priors on the embedding vectors
    # omega_loc should have shape [embed_dim, data.num_nodes]
    omega_loc = pyro.param('omega_loc', lambda: 0.5*torch.randn(embed_dim, data.num_nodes))
    # omega_scale should be a single positive tensor
    omega_scale = pyro.param('omega_scale', torch.tensor(1.0), constraint=constraints.positive)

    # Parameters governing the prior for the linear regression
    # beta_loc should be of shape [embed_dim]
    beta_loc = pyro.param('beta_loc', 0.5*torch.randn(embed_dim))
    # beta_scale should be a single positive tensor
    beta_scale = pyro.param('beta_scale', torch.tensor(1.0), constraint=constraints.positive)
    # mu_loc should be a single tensor
    mu_loc = pyro.param('mu_loc', torch.randn(1))
    # mu_scale should be a single positive tensor
    mu_scale = pyro.param('mu_scale', torch.tensor(1.0), constraint=constraints.positive)

    # Sample the coefficient vector and intercept for linear regression
    beta = pyro.sample('beta', dist.Normal(loc=beta_loc, scale=beta_scale*torch.ones(embed_dim)))
    mu = pyro.sample('mu', dist.Normal(mu_loc, mu_scale))

    # Subsample embedding vectors - this is going to be fun...
    with pyro.plate('nodes', size=data.num_nodes, subsample=node_ind) as plate_node_ind:
        omega = pyro.sample('omega', dist.Normal(
                                        loc=omega_loc,
                                        scale=omega_scale*torch.ones(embed_dim, data.num_nodes)).to_event(-1)
        ).index_select(-1, plate_node_ind)

    # Define plate for the edge subsampling to pass to model object
    with pyro.plate('edges', size=data.num_edges, subsample=edge_ind) as plate_edge_ind:
        # Note: we use pyro plate here as we need to keep the subsampling
        # persistent in both our model and guide functions, we so we
        # define an empty plate here just so we can use the same call
        # in the model.
        pass

    return omega, beta, mu


def model(data, node_ind, edge_ind, edge_list):
    r"""Defines a probabilistic model for the observed network data."""
    # Define priors on the regression coefficients
    mu = pyro.sample('mu', dist.Normal(torch.tensor(0.0), torch.tensor(2.0)))
    beta = pyro.sample('beta', dist.MultivariateNormal(
        loc=torch.zeros(embed_dim), covariance_matrix=4*torch.eye(embed_dim)
    ))

    # Define prior on the embedding vectors, do subsampling for the
    # embedding vector and then the likelihood term for the observed nodes
    with pyro.plate('nodes', size=data.num_nodes) as plate_node_ind:
        # Embedding vectors of shape [embed_dim, data.num_nodes]
        omega = pyro.sample('omega', 
                            dist.Normal(
                                loc=torch.zeros(embed_dim, data.num_nodes),
                                scale=omega_scale*torch.ones(embed_dim, data.num_nodes)).to_event(-1)
                            ).index_select(-1, plate_node_ind)

        # Draw Bernoulli, with or without data depending on if it is observed
        logit_prob = mu + torch.mv(omega.t(), beta)
        
        # Create mask corresponding to entries of ind which lie within data.nodes_train
        obs_mask = np.isin(ind_nodes.numpy(), data.nodes_train.numpy()).tolist()
        valid_data = data.gt[plate_node_ind].clone()
        valid_data[~obs_mask] = 0
        with poutine.mask(mask=obs_mask):
            pyro.sample('trust', dist.Bernoulli(logits=logit_prob), obs=valid_data)

    # Begin extracting the relevant components of the gram matrix formed by omega now
    # Note that to extract the relevant indices, we need to account for the
    # change in indexing induced by subsampling omega
    gram = torch.mm(omega.t(), omega)
    t = torch.zeros(node_ind.max() + 1, dtype=torch.long)
    t[node_ind] = torch.arange(len(node_ind))
    gram_sample = gram[t[edge_list[0, :]], t[edge_list[0, :]]]
        
    # Draw terms corresponding to the edges
    with pyro.plate('edges', size=data.num_edges, subsample=edge_ind):
        pyro.sample('a', dist.Normal(loc=gram_sample, scale=obs_scale*torch.ones(len(edge_ind))), obs=data.edge_weight_logit[edge_ind])
        

# Define SVI object
svi = SVI(model,
          guide,
          optim.Adam({"lr": .05}),
          loss=TraceGraph_ELBO())

In [107]:
# Define model object
def guide(data, node_ind, edge_ind, edge_list):
    r"""Defines a variational family to use to fit an approximate posterior
    distribution for the probability model defined in model."""
    ## Deleting arguments not used in the guide for linting purposes
    del edge_ind, edge_list
    
    ## Parameters governing the priors on the embedding vectors
    # omega_loc should have shape [embed_dum, data.num_nodes]
    omega_loc = pyro.param('omega_loc',
                           lambda: 0.5*torch.randn(embed_dim, data.num_nodes))
    # omega_scale should be a single positive tensor
    omega_scale = pyro.param('omega_scale',
                             torch.tensor(1.0),
                             constraint=constraints.positive)
    
    ## Paramaeters governing the prior fr the linear regression
    # beta_loc should be of shape [embed_dim]
    beta_loc = pyro.param('beta_loc',
                          0.5*torch.randn(embed_dim))
    # beta_scale should be a single positive tensor
    beta_scale = pyro.param('beta_scale',
                            torch.tensor(1.0),
                            constraint=constraints.positive)
    # mu_loc should be a single tensor
    mu_loc = pyro.param('mu_loc',
                        torch.randn(1))
    # mu_scale should be a single positive tensor
    mu_scale = pyro.param('mu_scale',
                          torch.tensor(1.0),
                          constraint=constraints.positive)
    
    ## Sample the coefficient vector and intercept for linear regression
    beta = pyro.sample('beta',
                       dist.Normal(loc=beta_loc, 
                                   scale=beta_scale*torch.ones(embed_dim)).to_event(1))
    mu = pyro.sample('mu',
                     dist.Normal(mu_loc, mu_scale).to_event(1))
    
    ## Handle the subsampling of the embedding vectors
    with poutine.scale(scale=data.num_nodes/len(node_ind)):
        omega = pyro.sample('omega', 
                            dist.Normal(loc=omega_loc[:, node_ind],
                                        scale=omega_scale).to_event(2))
    
subsample_dict = data.subsample(**sample_args_dict)
trace = poutine.trace(guide).get_trace(data, **subsample_dict)
trace.compute_log_prob()  # optional, but allows printing of log_prob shapes
print(trace.format_shapes())

Trace Shapes:             
 Param Sites:             
    omega_loc 2 6005      
  omega_scale             
     beta_loc      2      
   beta_scale             
       mu_loc      1      
     mu_scale             
Sample Sites:             
    beta dist      | 2    
        value      | 2    
     log_prob      |      
      mu dist      | 1    
        value      | 1    
     log_prob      |      
   omega dist      | 2 297
        value      | 2 297
     log_prob      |      


In [109]:
omega_model_scale = 1.0
obs_scale = 1.0
embed_dim = 2

def model(data, node_ind, edge_ind, edge_list):
    r"""Defines a probabilistic model for the observed network data."""
    # Define priors on the regression coefficients
    mu = pyro.sample('mu',
                     dist.Normal(torch.tensor([0.0]), 
                                 torch.tensor([2.0])).to_event(1))
    
    beta = pyro.sample('beta',
                       dist.Normal(loc=torch.zeros(embed_dim),
                                   scale=torch.tensor(2.0)).to_event(1))
    
    # Define prior on the embedding vectors, with subsampling
    with poutine.scale(scale=data.num_nodes/len(node_ind)):
        omega = pyro.sample('omega',
                            dist.Normal(loc=torch.zeros(embed_dim, len(node_ind)),
                                        scale=omega_model_scale).to_event(2))
    
    # Before proceeding further, define a list t which acts as the inverse
    # function of node_ind - i.e it takes a number in node_ind to its
    # index location
    t = torch.zeros(node_ind.max() + 1, dtype=torch.long)
    t[node_ind] = torch.arange(len(node_ind))
    
    # Create mask corresponding to entries of ind which lie within the
    # training set (i.e data.train_nodes)
    gt_data = data.gt[node_ind]
    obs_mask = np.isin(node_ind, data.nodes_train).tolist()
    gt_data[gt_data != gt_data] = 0.0
    obs_mask = torch.tensor(obs_mask, dtype=torch.bool)
    
    # Compute logits, compute relevant parts of sample
    if sum(obs_mask) != 0:
        logit_prob = mu + torch.mv(omega.t(), beta)
        with poutine.scale(scale=len(data.nodes_train)/sum(obs_mask)):
            pyro.sample('trust',
                        dist.Bernoulli(logits=logit_prob[obs_mask]).independent(1),
                        obs=gt_data[obs_mask])
    
    # Begin extracting the relevant components of the gram matrix formed by omega now
    # Note that to extract the relevant indices, we need to account for the
    # change in indexing induced by subsampling omega
    gram = torch.mm(omega.t(), omega)
    gram_sample = gram[t[edge_list[0, :]], t[edge_list[0, :]]]
    
    # Finally draw terms corresponding to the edges
    with poutine.scale(scale=data.num_edges/len(edge_ind)):
        pyro.sample('a', dist.Normal(loc=gram_sample, scale=obs_scale).to_event(1),
                    obs=data.edge_weight_logit[edge_ind])

subsample_dict = data.subsample(**sample_args_dict)
trace = poutine.trace(model).get_trace(data, **subsample_dict)
trace.compute_log_prob()  # optional, but allows printing of log_prob shapes
print(trace.format_shapes())

Trace Shapes:          
 Param Sites:          
Sample Sites:          
      mu dist |   1    
        value |   1    
     log_prob |        
    beta dist |   2    
        value |   2    
     log_prob |        
   omega dist |   2 270
        value |   2 270
     log_prob |        
   trust dist |  28    
        value |  28    
     log_prob |        
       a dist | 166    
        value | 166    
     log_prob |        


In [110]:
# Define SVI object
svi = SVI(model,
          guide,
          optim.Adam({"lr": .05}),
          loss=TraceGraph_ELBO())

In [111]:
t0 = time.time()

pyro.clear_param_store()
num_iters = 200
sample_args_dict = {'sample_str': 'p-sampling', 'sample_args': {'sample_prob': 0.005}}

for i in range(num_iters):
    subsample_dict = data.subsample(**sample_args_dict)
    elbo = svi.step(data, **subsample_dict)
    if i % 5 == 0:
        logging.info("Elbo loss: {}".format(elbo))
        if (i > 0):
            t1 = time.time()
            logging.info("Time to run 5 gradient steps of SVI: {}".format(t1-t0))
            t0 = time.time()

Elbo loss: 334435.5
Elbo loss: 137262.75
Time to run 5 gradient steps of SVI: 0.7655518054962158
Elbo loss: 90147.3828125
Time to run 5 gradient steps of SVI: 0.6839547157287598
Elbo loss: 85527.5234375
Time to run 5 gradient steps of SVI: 0.7819507122039795
Elbo loss: 67257.734375
Time to run 5 gradient steps of SVI: 0.6419391632080078
Elbo loss: 72872.8359375
Time to run 5 gradient steps of SVI: 0.6312680244445801
Elbo loss: 67029.578125
Time to run 5 gradient steps of SVI: 0.6811816692352295
Elbo loss: 67685.296875
Time to run 5 gradient steps of SVI: 0.6515693664550781
Elbo loss: 60318.62109375
Time to run 5 gradient steps of SVI: 0.6661176681518555
Elbo loss: 62692.75
Time to run 5 gradient steps of SVI: 0.6082806587219238
Elbo loss: 59708.640625
Time to run 5 gradient steps of SVI: 0.6349401473999023
Elbo loss: 59145.921875
Time to run 5 gradient steps of SVI: 0.6262016296386719
Elbo loss: 69573.078125
Time to run 5 gradient steps of SVI: 0.6039650440216064
Elbo loss: 69402.97656